In [1]:
import base64
import numpy as np
import os
import pandas as pd
import time

from PyQt5.QtCore import *
from PyQt5.QtGui import *
from PyQt5.QtNetwork import *
from PyQt5.QtWidgets import *
from PyQt5.QtMultimedia import *
from PyQt5.QtMultimediaWidgets import *

from sklearn.linear_model import LinearRegression
from scipy.optimize import curve_fit

import matplotlib
import matplotlib.pyplot as plt
from matplotlib.backends.backend_qt5agg import FigureCanvasQTAgg, NavigationToolbar2QT as NavigationToolbar
from matplotlib.figure import Figure
from matplotlib import gridspec
from mpl_toolkits.mplot3d import Axes3D
matplotlib.use('Qt5Agg')

from watchdog.observers import Observer
from watchdog.events import FileSystemEventHandler

In [2]:
class MplCanvas(FigureCanvasQTAgg):

    def __init__(self, parent=None, width=5, height=4, dpi=100, is3d=False):
        fig = Figure(figsize=(width, height), dpi=dpi)
        self.axes = fig.add_subplot(111, projection='3d' if is3d else None)
        super(MplCanvas, self).__init__(fig)

class FileChangeHandler(FileSystemEventHandler, QObject):

    file_created = pyqtSignal(str)
    file_deleted = pyqtSignal(str)

    def on_created(self, event):
        if not event.is_directory:
            self.file_created.emit(event.src_path)

    def on_deleted(self, event):
        if not event.is_directory:
            self.file_deleted.emit(event.src_path)

class FileMonitorThread(QThread):

    file_created = pyqtSignal(str)
    file_deleted = pyqtSignal(str)

    def __init__(self, folder_path):
        super().__init__()
        self.folder_path = folder_path
        self.observer = None
        self.is_running = True

    def run(self):
        event_handler = FileChangeHandler()
        event_handler.file_created.connect(self.file_created)
        event_handler.file_deleted.connect(self.file_deleted)

        self.observer = Observer()
        self.observer.schedule(event_handler, path=self.folder_path, recursive=True)
        self.observer.start()

        try:
            while self.is_running:
                time.sleep(0.5)
        except KeyboardInterrupt:
            pass
        finally:
            self.stop()

    def stop(self):
        if self.observer is not None:
            self.observer.stop()
            self.observer.join()
        self.is_running = False


class MainWindow(QWidget):

    global driver
    
    def __init__(self, points_xy_path):
        super(MainWindow, self).__init__()

        # Load saved settings
        self.logdir = None
        os.makedirs(f'./logs', exist_ok=True)
        if os.path.exists('logs/logdir.txt'):
            with open('logs/logdir.txt', 'r') as f:
                logdir = f.read()
                if os.path.exists(logdir):
                    self.logdir = logdir

        self.points_xy_path = points_xy_path
        self.monitor_thread = None
        self.data_points = {}
        self.num_points = 0
        self.scatter_plot = None
        self.plane_plot = None
        self.initUI()
        self.update_plot()

    def initUI(self):

        # Plotter Box
        self.canvas = MplCanvas(self, width=5, height=4, dpi=100, is3d=True)
        self.ax = self.canvas.axes
        self.elevation = 30
        self.azimuth = -60
        self.ax.view_init(self.elevation, self.azimuth)
        self.toolbar = NavigationToolbar(self.canvas, self)

        self.plotter_box = QVBoxLayout()
        self.plotter_box.addWidget(self.toolbar)
        self.plotter_box.addWidget(self.canvas)

        self.plotter_widget = QWidget()
        self.plotter_widget.setLayout(self.plotter_box)

        # Console
        self.text_edit = QTextEdit()
        self.text_edit.setFixedHeight(50)
        self.text_edit.setReadOnly(True)

        # Logging directory selection
        logdir_box = QHBoxLayout()
        self.logdir_button = QPushButton("Select Directory")
        self.logdir_button.setFixedWidth(100)
        self.logdir_button.clicked.connect(self.select_logdir)
        self.logdir_edit = QLineEdit()
        if self.logdir is not None:
            self.logdir_edit.setText(self.logdir)
        else:
            self.logdir_edit.setPlaceholderText("Select a directory to log")
        self.logdir_edit.setReadOnly(True)
        self.rmvexisting_button = QPushButton("🗑️")
        self.rmvexisting_button.setFixedWidth(20)
        self.rmvexisting_button.clicked.connect(self.remove_ext)
        logdir_box.addWidget(self.logdir_button)
        logdir_box.addWidget(self.logdir_edit)
        logdir_box.addWidget(self.rmvexisting_button)

        # Logging directory operations
        logops_box = QHBoxLayout()
        self.monitor_button = QPushButton("Start Logging")
        self.monitor_button.clicked.connect(self.start_logging)
        self.rmvlast_button = QPushButton("Remove Last Point")
        self.rmvlast_button.clicked.connect(self.remove_last)
        self.rmvall_button = QPushButton("Remove All Points")
        self.rmvall_button.clicked.connect(self.remove_all)
        logops_box.addWidget(self.monitor_button)
        logops_box.addWidget(self.rmvlast_button)
        logops_box.addWidget(self.rmvall_button)

        # Commands box
        cmd_box = QVBoxLayout()
        cmd_box.addWidget(self.plotter_widget)
        cmd_box.addWidget(self.text_edit)
        cmd_box.addLayout(logdir_box)
        cmd_box.addLayout(logops_box)


        # Points Table
        self.table = QTableWidget()
        self.table.setColumnCount(4)
        self.table.setHorizontalHeaderLabels(['Point', 'X', 'Y', 'Z'])
        self.table.horizontalHeader().setSectionResizeMode(QHeaderView.Stretch)
        self.table.verticalHeader().setVisible(False)
        self.table.cellChanged.connect(self.update_plot)

        # Populate the table with points xy data
        pointsdf = pd.read_csv(self.points_xy_path)
        pointsdfnp = pointsdf.to_numpy()
        self.table.setRowCount(pointsdfnp.shape[0])
        for i in range(pointsdfnp.shape[0]):
            self.table.setItem(i, 0, QTableWidgetItem(str(pointsdfnp[i, 0])))
            self.table.setItem(i, 1, QTableWidgetItem(str(pointsdfnp[i, 1])))
            self.table.setItem(i, 2, QTableWidgetItem(str(pointsdfnp[i, 2])))

        # Export button
        self.export_button = QPushButton("Export Data")
        self.export_button.clicked.connect(self.export_table_data)

        # Table box
        table_box = QVBoxLayout()
        table_box.addWidget(self.table)
        table_box.addWidget(self.export_button)

        box = QHBoxLayout(self)
        box.addLayout(cmd_box)
        box.addLayout(table_box)
        box.addLayout(self.plotter_box)
        self.setLayout(box)

        icon_bytes = base64.b64decode(b'iVBORw0KGgoAAAANSUhEUgAAAIAAAACACAMAAAD04JH5AAABnlBMVEVHcEyAgIB+fn5+fn5+fn6AgIB/f3+EhISAgICBgYF9fX2AgIB9fX19fX2AgIB/f39/f39+fn5+fn5/f39+fn6AgID////krVzlyVz+/v6mpqb9/f2Hh4e/v7/J1lzkklvAwcH5+fnPz87X19fx8fGMjIyRkZGKioro6Oi1tLXc3N3TuEvg4OBc1q3u7u6oqKj39/ewsLDq6uvXhVDT09L7+/vLy8uUlJT09PSamprExMLj4+SioqJKxZydnp6EhIS7vbzMekS5ubnHx8aPj4/hqlrPl0aXl5ekpKW9tqi3t7jBvbPCrn3ix1uqqqlLvJjMum/JpXHF0VzaqmLbpFPXwGTVnkzfkl+6o5HQtUm9qIq/mYHbwFK3t5u7spfexV9exsbNskW5raDNlG6/ymaet7K0u3rTqGiAgICxtoqyv0i4wW6PtaqnoZmrvbnXkmW7yE5wwaa8sox2dsdxuLhlyaeWi26wj3NpppauhVpf0KrUgkt/u6fBiGCCsrJnZ6xUzqWpm4aVkYGDnpavpWSIhH2emXt3d5nErlXToVdCceMxAAAAFnRSTlMAFmfdzydJ+AvrxTCzlR54WYinOoIFpiZIZgAAD41JREFUeNq8WflP22oWfWQlIeytIbGJEtuJF6zEjmI7iSM7IqZkgYiwVAUBVSvEUoZR1Up03g+jUXnd5r+ez0sSf58DBV4794ckBuJ7fO85516bP/54WjyfnAkvRiJzICKRxfDM5PM//m8xGY4EphPxYCwWdSMWC8YT04FIePL3J58KTMdjUbOa4jIKxQuarmsCT5EZLlU1o7H4dGDqN4KYiMzHY0xVUviCiGNOpFLuB1wsCIqkM7H4fGTit1z74nwoqkokK2KewFMp3HsssqSkRkPzi7+6DhOBRCwvUXQSgwMFACJJU1w+lgjM/ML04dkQ0SJpzB9jAFhBky0i9Cz8q9I/C+YkQcawhwPAMFmQcsFfAmFiNpjrsEkMexwA0Aq2kwvO/l0+Ts6FGI7F7ox7AIBgOSY097foOLUQTQlJ7KkAsKSQii5MPf3yA0GVzGLY0wFgWJZUg4EnFiG8EJUKY+hV4MvigwFgWEGKLjyJjJFQnvRRP8uS7cP+q9IjAGA4mQ9FHl/+2VgLJR9esrLvLi31qccAAGRsxWYf2YaJ6WhHRC5e6NrZQZw3hyxLpZI/B5AUO9HpRwlyJsEocPmzfHvzfMmN3XZWLAnlZoOTTFPiGs2yUBLvrQSuMIlHeHM4blIw8UD63UH6pfW9z62NHMGYalVnGL2qmgyR2yhmePoeEJQZfzAVp0KqAPewO0q/vvfyxYs/qx2FZ+mKKLZaolihWV7p1E3CLCol+S4Eghqaemh+BspfUQ7PvdlfvNzrk34OyDSfqTO5YrlyFwLmYQjCIYZY8VyH1u6vD9KD7HvWwW43OVYFWbapE2qmNHY8rRBM6AFdmImrwsoIQZY8dKu//tK6dvfziXiXDLO8xJiNwrj8K4Ia/ykTJxKmYP+xg4Du9kdX/3J9yMPD0t0+YI1Bs1bx55cxwUz8RI2T0ww1/HNwrpqbH04/sqI7jIiViCqF+/IDLTDT9zvSbFSBvlA62XWrv7fkjYEV3eWESb5OcLQvP4Yp0dl7/T/WkeGvCIfr1uXvrUP5gRXJP7FisclUeV9+TO7EIvcJoCX6ira5h16+TQL6p7NA05mmjOQHwFp3S2FyIc/62lbR/3y57su/tCmMASCTvHeAVDiCE5H8gB/5hbtoEIiSKHFxusdcn/vzL7lWBANgL8+Ovcuz3CR6NA7nxzAyGrjDAYOSjEqHq+fKdHsMAteKYADKVXrn6ljxQCjn6h0kP4ZLwanxDVBR+8A7FwTQG3uy6wPgWhE0jsXjnXTagjDamDCKuOBQlhTUsU2Ygxtg21CdsOHzh34WACtKZkW6XqfFrItBOEvbsXPW1pLDIhJ13x0NGZ0bY4GhFLp/ir0c5TSwvOknwVqjp6t5wzBVvdcoW7eMtdO0G6eXijigEZXriehilwr5DXEWHoHWtzmi7FJYHjjiaCj++6talxoZVc00pLrKMNUOeZkexs7lyFDLBCf7BqPPjsJBDt2tmkRzeBKxe+5Jb43k/76l5cHtuVyxboqJL+93hiXoevTnnAgySi6ImsGzHLqC8gznsZHCW2gorrtW5FFBoffu3TsXwulxwaN/mWN4dFjknqEF6KAErOoVryOxr3bhoWhbkQdA6TK98x5AsBugQf5T0asoETtICXwFwDlGgz3RkgK4/KEt2lbkAQBMIG1BAEU4K8P+l9QYVItsbhaWgIQwgCJqqCuT8FSwrWgEwDGBdBoU4aqGo/5bIyiEBRIkhACBSKBSrYs+V76GpoJtRSMAAxNI77z7yomo/4r1KrKhCITHkCcTLUQoNYJHXXlNQqbCIesBkByawM7xGiGtof7Leytqn7GVGNnhYgwxwYKJtgRfuzDI0ttdZCsaAqDdDgACChhpXKyh/iuZiNGTscUhgPk8QtKGT5SiZBgruOb1ZGsrGgKgBh04I0G5DEMSfcJrIDLLz48oyMG/K5kpBJG8RpBWW6lDeCsaAJC7bgeuarJFGJJYQ92vYyLb+oiGkShC0YxxfQw/FFNAC60TZ5U+tBUNAJRcGz5tVxz+1QgFLQGTQYQWHSxn8yp8vZWN/Obr7RPPXNdy1qpgIRC7fe9WNABAXrkELLn8l6WchgpPhYVAq24PJuMI48rE9f7ysgWhMBDRBj1Qo2c9sazIeVQrth0KXvJD/dEbdYQGPJhtMKK4o4MpRANyz/ywtQxia/ukWUra04Qa+cFoPdlta1S1Sml0UjtzCejxHwodQVm9KCM6cDajAANTvpSrbi87sbX9oVbCCnkJ9ziSu56AsXDDMIbBMGaqZ1Pwqit6/AeX8ojwmrkSwgrHi6Z1uFaK3QEPBMlkIU8kN92h+KlpPa6nMvqNNQFO2wXI/3zCYxmYmGJ12qUAbDm9vNOBQWzfXGu4F0Gj1neGYp93OECdWWPwWMvA/tdBvCdbL8L25JAgHFMQg9A3vfm3jt7sf+hqsndXdqfCeTNpqQAHJrDz/qbXQPyXZRD7XTFhuSmxsO0CAkLW1D4E4GB1eWt/szt8XC03LozPu+6zIguAbQJXdeOige7fVRE5NTxhBNsJAkihMrm3r70Ajt4cWW8AAu9srXjDMFr2erJ0WLAAWCZw1e0YRsM30+GEtJlBRo7FwmkYJt5TYQqsHriH+5tt+86rTKysENVDx4qKqWQWmMDpsWT9tIwMdRXes+R6D/ex8HkC+afLBkIB0AHrzXp5vdmmxIJazOIrhmk5Yl8ppjBgAjuXLdD/bBG9tUF70NlA/uGTeA5EwCGDqLUNAbA6sOWBULQmJRh4ub61FbVSePM0faba/PMJj0QsRkGcgAMymAlmkJW9tw1rYGtYAevl6Mtn6zkYQHB7DrYivQg2gauc4fAfFR7LlBEWwoSvBWeACmEjLjM9SARHFgWGFbA4ub9tjUrQhdvdpcN8jzo7vXXzYxpiNRW1cS8gS4eLMQrxy7cQgIPVASVtIFurq86couWM8W29T6S6V98G+eVsPSUjrEvCMoDnAwW2okgUlkojD6nQ4eDW8paXEg6E0orx6ZzYuHTy47TQ5AoZxGokPXtfRXhgBKgPceoxDODITe50wnUFe1TWesanW+LWyMhZluy0Pn78qFEMvAU0VBHWHedzormohuGjSEobJ69REQwJ4AFgz6n8168XFxKVKX50QtAYyns2LJOnk56Ti7qU9P5aA/fpc1E95Q2T+Haw6ok30NHqgff44MvNxcVF7vv3/zjxXa8bVehsqlH3HrbA6PaG/mQABwc/fnz58vkWFIBoFYeR0p8CAG3B8ZgWeCfD/v725od2l6QUyTQMg2iIyVFNH9+CB5Bw8HHrNUh986rbBFuYnNUaOmNwf+lrX6XRsvkEEqIyXMm3x8jQSv3hpFsrU6Y98mThn//Q80aG/ddfBe7memi4T5Chz4jM9j4E4Mf2/5q3lt7EkSCslfa+NyTWjgA/BsfiYcA2xtgMDkiTkNU4N5CsHSEQN0KEBANJACWTP77dbYNN2+DHZpRYOUXI/bm6uqr6q6/u/vlxo1KFqiJBWkyTUjl+tttNQP5J0+t1Vbl/fOXTTuCJH4j8ofjyzrE42uzXXyrPKe5r5XwBLL/YdRD/p5prPsW9Gis5lzQU+5PR5Z1t8X/RZstYQuPYt8VwMbObGtLMNMGS/MrY3jIJklEWJCNfOmYrP4DFxYKjE+HyXgsp9GRtDncyY9efzFQ3gU+k5a2xHXHJ0rG/IKmUPBYHjusWzYo4WQyH6zf+QOZvdHMCjJ+7BQgGYqKCxFeSlbXj2rm3v9IxFFx+uEPFl53/qLmuT6HtmdHWqK2+UglKMn9Rip0k3r6XOcsvJgUJFKX2+kR7qesbdPK5Qc1wHCFuUYpFojReO4M9IOBacPnhTgWlCCjLL5DTC5MnXZ/bfiauDKM2HjXjlOU8Kst9F5PuBcYXIU+iFvDzecR/tdtkGf6vuinq+hIdba78dVszag8DT68q4sXEdzUraznMUHAnS9PhLss4lF1arLO9JkHPAYCnGSE1e2yduh0DBH2rI8S8mvkvp9jZsRmjdBZ+/oH/K7W7eU1bvuj6y7StoVYlM3oACDLj21K8y2nA9Ry72NvXc6Z03H8qydekCZ9192+byWgO+giB7QjRr+c+gqLcyuG8Oe3rv6FdeYEWmIv7eE9bGYjgYSAScQiKAIoGI7e9FI3nHYVNEfhAcX7AT3TGCEHfUoUYFI2fpKrjNKVLUnm/gZojAEv3cAu3NoLM+KbcjUxSBdB0uBseaDrv+iAMIQBPs8OOpUujBxvB9udVdJrujz8rOFGJdw+kK1LG+38gDCEAxanHgMARIQLwNxAljKgsnSIqI1O1x28EYcgGsPH8OC1aGYDAeIaOoESkagPJasJPVmPcJwhDNoC5dwcJFboB3AXgCFVPWD5HVkek60n16FedpQNgeXTehJut8Yz8AJxHPhpdH61hIVyRFW9jePbkAADB2Gva8s9aJuMgsBw1WEjDIlrLJiXJ7Dc31zDT4h7AxAVL0N+6V8ARHQQZ60KJ0rKJ0rRCPq6RFc4ThmwAxWnJo93QminoiHsE4xsOFqshTasIbbu9LuIL2+OIfRhyAGwcMQHXY7/AZCgBRzwgAI7AhLftQhuX7gEBafA7DbDBMOQAQDWJQrv6HeXmwUXQt1psaOMyrHV7dEbVBlmvyNOnA4BlG7VuG+phK6rQDfYIao/3TGjr9nzzGo9KfK9BrnXwvJgmSIhmHjavj+QbBeuAoGY8j0dcWPP6fPs+QCvaWZvu8yZzeJOKsjI2ArA+dASvrCiwfX9OwBAoFaWys8lk+vY2ncw6tBQg3RrbCOD60BHoEAHDGQnHSZViTlC0a0UKljLtHXF/FqphEo5AEUtPY8WzYs0zkk4OJUY7KvcHrodJJ0Qsp2Q8ZEdKqCvmLbT/qFb2GFI8JeMJFjIJl+SlkgwA03p8tv1gRUUSMkEpl+KLv1In32omAdBssffIEfsrTymhXJ9TNfrFbMiUjbwqxAUgqPkGD64K8ABQUcVsfjmfY8tLUuOJOAD2cj7giP0BHV3Ohwsa3STbcNNgBACuoJG2kP9HFjTikk63QlG7QXMOoZJOgq7GknT6RK3HabDCC2EA/q+o1S/r9UDI1smWyuVOA3gPWa9f2OzdCLHM5rULnpH8kVBi+It3ETYHSLu9n1yVy12yq/1OaXeAuP0YA8Nnv/9WcXuQvB8HEVPeL8WT99sDDkoq/Ik24JBS4g44BI94xE1GblyKP+JxasglCQAp0ZDL6TGfuAASj/l8/KDTx496fYJht48f9/sEA4/JRz7/eq/lEw69vvfs7weP/X6CwefPMPr9GYbf33/8/z+WaX62sYUE9QAAAABJRU5ErkJggg==')
        icon = QImage.fromData(icon_bytes)
        icon_pixmap = QPixmap.fromImage(icon)
        self.setWindowIcon(QIcon(icon_pixmap))

        QApplication.setStyle(QStyleFactory.create('Fusion'))
        self.setWindowTitle("Confocal Data Logger")
        self.show()

    def select_logdir(self):
        logdir = QFileDialog.getExistingDirectory(self, "Select Directory")
        if logdir:
            self.logdir_edit.setText(logdir)
            self.logdir = logdir
            with open('logs/logdir.txt', 'w') as f:
                f.write(logdir)

    def start_logging(self):
        if self.logdir is None:
            QMessageBox.warning(self, "Warning", "Please select a directory to log.")
            return
        if self.monitor_thread is None or not self.monitor_thread.isRunning():
            self.monitor_thread = FileMonitorThread(self.logdir)
            print(f"Monitoring {self.logdir}")
            self.monitor_thread.file_created.connect(self.on_file_created)
            self.monitor_thread.file_deleted.connect(self.on_file_deleted)
            self.monitor_thread.start()
            self.monitor_button.setText("Stop Logging")
        else:
            self.monitor_thread.stop()
            self.monitor_button.setText("Start Logging")

    def on_file_created(self, file_path):
        if not file_path.endswith('.csv'):
            return
        csv_path = f"{os.path.dirname(file_path)}/{os.path.basename(file_path)}"
        time.sleep(0.25)
        print(f"File created: {csv_path}")
        self.num_points += 1
        mean, std = self.get_data(csv_path)
        self.data_points[f'Point {self.num_points}'] = {'Path': csv_path, 'Mean': mean, 'Std': std}
        self.text_edit.append(f'Added Point {self.num_points}: {mean:.3f} ± {std:.3e}')
        self.table.setItem(self.num_points-1, 3, QTableWidgetItem(f'{mean:.3f}'))
        self.update_plot()

    def on_file_deleted(self, file_path):
        if not file_path.endswith('.csv'):
            return
        csv_path = f"{os.path.dirname(file_path)}/{os.path.basename(file_path)}"
        time.sleep(0.25)
        print(f"File deleted: {csv_path}")

    # Remove last logged file
    def remove_last(self):
        if self.logdir is None:
            QMessageBox.warning(self, "Warning", "Please select a directory to log.")
            return
        if self.num_points == 0:
            QMessageBox.warning(self, "Warning", "No points to remove.")
            return
        point = self.data_points.pop(f'Point {self.num_points}')
        os.remove(point['Path'])
        self.num_points -= 1
        self.text_edit.append(f'Removed Point {self.num_points + 1}: {point["Mean"]:.3f} ± {point["Std"]:.3e}')
        self.table.setItem(self.num_points, 3, QTableWidgetItem(''))
        self.update_plot()

    # Remove all logged files
    def remove_all(self):
        if self.logdir is None:
            QMessageBox.warning(self, "Warning", "Please select a directory to log.")
            return
        confirmation = QMessageBox.question(
            self,
            "Confirmation",
            "Are you sure you want to remove all points?",
            QMessageBox.Yes | QMessageBox.No,
            QMessageBox.No
        )
        if confirmation == QMessageBox.Yes:
            for point in range(self.num_points, 0, -1):
                self.remove_last()

    # Remove all files in logdir
    def remove_ext(self):
        if self.logdir is None:
            QMessageBox.warning(self, "Warning", "Please select a directory to log.")
            return
        confirmation = QMessageBox.question(
            self,
            "Confirmation",
            "Are you sure you want to remove all files?",
            QMessageBox.Yes | QMessageBox.No,
            QMessageBox.No
        )
        if confirmation == QMessageBox.Yes:
            for file in os.listdir(self.logdir):
                os.remove(os.path.join(self.logdir, file))

    # Get mean and std from csv file
    def get_data(self, csv_path):
        df = pd.read_csv(csv_path, skiprows=2)
        dfnp = df.to_numpy()
        mean = np.mean(dfnp[:,1])
        std = np.std(dfnp[:,1])
        return mean, std

    # Get xyz data from table
    def get_table_data(self):
        values = []
        for i in range(self.table.rowCount()):
            if self.table.item(i, 3) is None or self.table.item(i, 3).text() == '':
                continue
            for j in range(self.table.columnCount()-1):
                values.append(float(self.table.item(i, j+1).text()))
        self.table_data = None
        if len(values) != 0:
            self.table_data = np.array(values).reshape((self.num_points, 3))
            self.table_data[:,2] = -self.table_data[:,2]

    # Export xyz data to csv
    def export_table_data(self):
        self.get_table_data()
        if self.table_data is not None:
            file_path, _ = QFileDialog.getSaveFileName(self, "Export XYZ Data", "", "CSV Files (*.csv)")
            if file_path:
                np.savetxt(file_path, self.table_data, delimiter=",")
                self.text_edit.append(f'Exported data to {file_path}')

    # Update plot
    def update_plot(self):

        self.get_table_data()
        if self.table_data is not None and self.table_data.shape[0] >= 3:
            xx_ = self.table_data[:,0]
            yy_ = self.table_data[:,1]
            zz_ = self.table_data[:,2]

            model = LinearRegression()
            model.fit(np.c_[xx_, yy_], zz_)

            # Find plane equation
            coef_x, coef_y = model.coef_
            intercept = model.intercept_
            print(f"\nPlane equation: z = {coef_x:.3e}x + {coef_y:.3e}y + {intercept:.3e}")

            # Find normalised normal vector
            magnitude = np.sqrt(coef_x**2 + coef_y**2 + 1)
            normal = np.array([coef_x, coef_y, -1])/magnitude
            print(f"|Normal| vector: {normal}")

            # Find pitch (Rx) and yaw (Ry) angles
            pitch = -np.arctan(normal[1]/normal[2])*180/np.pi
            yaw = -np.arctan(normal[0]/normal[2])*180/np.pi
            print(f"Pitch (Rx): {pitch:.3f}°  Yaw (Ry): {yaw:.3f}°")

            # Generate plane for plotting
            xp_ = np.linspace(0, 190, 10)
            yp_ = np.linspace(0, 290, 10)
            xxp, yyp = np.meshgrid(xp_, yp_)
            zzp = coef_x*xxp + coef_y*yyp + intercept

        if self.scatter_plot is not None:
            try: self.scatter_plot.remove()
            except Exception as e: print("Ignored")
        if self.plane_plot is not None:
            try: self.plane_plot.remove()
            except Exception as e: print("Ignored")

        if self.table_data is not None:
            self.scatter_plot = self.ax.scatter3D(self.table_data[:,0], self.table_data[:,1], self.table_data[:,2], c=self.table_data[:,2], cmap='viridis')
            if self.table_data.shape[0] >= 3:
                self.plane_plot = self.ax.plot_surface(xxp, yyp, zzp, alpha=0.5, color='red')
                self.ax.set_title(f'TP504 Profile\nPitch: {pitch:.2f}°, Yaw: {yaw:.2f}°')
            else:
                self.ax.set_title('TP504 Profile')
        else:
            self.ax.set_title('TP504 Profile')

        self.ax.set_xlabel('X')
        self.ax.set_ylabel('Y')
        self.ax.set_zlabel('Z')

        PADDING = 30
        self.ax.set_xlim(0-PADDING, 190+PADDING)
        self.ax.set_ylim(0-PADDING, 290+PADDING)
        self.ax.set_zlim(-1.75, 0.25)
        self.ax.invert_xaxis()

        xy_scaler = 290/190
        self.ax.set_box_aspect((1, xy_scaler, 0.5))
        self.canvas.draw()

    def closeEvent(self, event):
        if self.monitor_thread is not None and self.monitor_thread.isRunning():
            self.monitor_thread.stop()
        QApplication.quit()


app = QApplication.instance()
if app is None: app = QApplication([])
window = MainWindow(points_xy_path='points/gap_points_xy.csv')
app.exec_()

0